In [25]:
import networkx as nx
import os
import numpy as np
from sklearn import preprocessing

In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [3]:
# gpk_path = "/h/diya.li/dev/dscompetition/data/gpk"
# for i in os.listdir(gpk_path):
#     gpk = os.path.join(gpk_path, i)
#     break
# G = nx.read_gpickle(gpk)
# A = nx.adjacency_matrix(G)
# options = {
#     'node_color': 'blue',
#     'node_size': 1,
#     'width': 1
# }
# nx.draw(G, with_labels=False, **options)

In [1]:
import pandas as pd
from tqdm import tqdm

In [6]:
abstract_path = "/h/diya.li/dev/dscompetition/data/abstract.csv"
ab_df = pd.read_csv(abstract_path, index_col=None)
# ab_df.columns = ab_df.iloc[0]
# ab_df.drop(ab_df.index[0])
# ab_df.to_csv(abstract_path, header=None)
df = ab_df[['id','publish_year','author_list', "keywords_from_api", "abstract"]]
df = df.dropna(axis=0)
df.head()

,id,publish_year,author_list,keywords_from_api,abstract
1,100233SE,2016,"[601005549, 601004479, 902008143]","['Amino Acids', 'Health', 'Nutrition', 'Argina...",L-arginine (Arg) is utilized via multiple path...
2,100233SE,2016,"[601005549, 601004479, 902008143]","['Amino Acids', 'Health', 'Nutrition', 'Argina...",L-arginine (Arg) is utilized via multiple path...
3,100234SE,2016,"[601005549, 601004479, 902008143]","['Endothelial Cells', 'Ornithine', 'Putrescine...",Polyamines are essential for proliferation of ...
4,100234SE,2016,"[601005549, 601004479, 902008143]","['Endothelial Cells', 'Ornithine', 'Putrescine...",Polyamines are essential for proliferation of ...
5,100235SE,2014,"[607000169, 416005337, 919004726, 601004479]","['Complementary And Integrative Health', 'Nutr...",BACKGROUND: Extracts of Russian Tarragon (RT) ...


In [7]:
def process_features(x):
    a = str(x)
    features = nlp(a).vector
    if len(features) != 96:
        # make sure keep the same dimension
        raise BaseException("dimension error")
    else:
        p.update(1)
        return ",".join([str(x) for x in features])
with tqdm(total=df.shape[0]) as p:
    df['comma_separated_list_of_features'] = df['abstract'].apply(lambda x: process_features(x))

100%|██████████| 13375/13375 [06:46<00:00, 32.92it/s]


In [8]:
df['keyword'] = df['keywords_from_api'].apply(lambda x: x[1:-1].replace("'", "").split(","))
df['author'] = df['author_list'].apply(lambda x: x[1:-1].split(","))

df = df.explode('keyword', ignore_index=True)
df = df.explode('author', ignore_index=True)
author_dict = {au:idx for idx, au in enumerate(df['author'].unique())}
keyword_dict = {au:idx for idx, au in enumerate(df['keyword'].unique())}
id_dict = {au:idx for idx, au in enumerate(df['id'].unique())}

df = df[df['publish_year'] != 901006572]
df = df[df['publish_year'] != 322009873]

In [9]:
def process_user_id(x):
    return author_dict[x]
df['user_id'] = df['author'].apply(lambda x: process_user_id(x))

def process_item_id(x):
    return id_dict[x]
df['item_id'] = df['id'].apply(lambda x: process_item_id(x))

def process_state_label(x):
    # return keyword_dict[x]
    return 0 # State label should be 1 whenever the user state changes, 0 otherwise. If there are no state labels, use 0 for all interactions.
df['state_label'] = df['id'].apply(lambda x: process_state_label(x))

def process_timestamp(series):
    l = series.apply(lambda x: int(x)).values.reshape(-1,1)
    scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(l).T[0]
df['timestamp'] = process_timestamp(df['publish_year'])

df = df[["user_id", "item_id", "state_label", "timestamp", "comma_separated_list_of_features"]]

In [12]:
df.to_csv("scholar.csv", index=False)

In [20]:
ls = []
with open("scholar.csv", "r") as f:
    for idx, l in enumerate(f.readlines()):
        ls.append(l.replace("\"", ""))
with open("scholar_.csv", "w") as ff:
    ff.writelines(ls)

In [2]:
template_path = "/h/diya.li/dev/dscompetition/scholar.csv"
df = pd.read_csv(template_path, index_col=None)
df.head()

,user_id,item_id,state_label,timestamp,comma_separated_list_of_features
0,0,0,0,0.877551,"0.100074254,0.1365942,-0.038670383,-0.00133076..."
1,1,0,0,0.877551,"0.100074254,0.1365942,-0.038670383,-0.00133076..."
2,2,0,0,0.877551,"0.100074254,0.1365942,-0.038670383,-0.00133076..."
3,0,0,0,0.877551,"0.100074254,0.1365942,-0.038670383,-0.00133076..."
4,1,0,0,0.877551,"0.100074254,0.1365942,-0.038670383,-0.00133076..."


In [40]:
df = df.drop_duplicates()

In [46]:
df['user_id'] = df['user_id'] + 1
df['item_id'] = df['item_id'] + 1

In [28]:
print(len(df['user_id'].unique()))
print(len(df['item_id'].unique()))
print(len(df['state_label'].unique()))
print(len(df['timestamp'].unique()))
print(len(df['comma_separated_list_of_features'].unique()))

11503
11509
11507
11510
11507


In [29]:
df.dtypes

user_id                             float64
item_id                             float64
state_label                         float64
timestamp                           float64
comma_separated_list_of_features    float64
dtype: object